#### Carregando bibliotecas

In [ ]:
import os
import glob
import json
import openpyxl
import pandas as pd
import pandas.io.json as pd_json

#### Funções auxiliares

In [ ]:
def listar_arquivos(extensao='json'):
    search = "output/*.{}".format(extensao)
    files = glob.glob(search)
    files.sort()
    return files

In [ ]:
def ler_json(caminho):
    with open(caminho) as fp:  
        dados = json.load(fp)
    return dados

In [ ]:
def obter_os_contrato(nome_arquivo):
    nome_sem_extensao = nome_arquivo.split('.')[0]
    partes = nome_sem_extensao.split('_')
    return int(partes[1]), int(partes[3])

In [ ]:
def obter_codigo_nome_os():
    arquivo = 'input/contratos-controle.xlsx'
    df = pd.read_excel(arquivo, sheet_name='contratos')
    cod_os_nome = {}
    for os in df['ORGANIZACAO']:
        codigo_nome = os.split(' - ')
        cod_os_nome.update({int(codigo_nome[0]): codigo_nome[1]})
    return cod_os_nome

In [ ]:
def obter_codigo_numero_contrato():
    arquivo = 'input/contratos-controle.xlsx'
    df = pd.read_excel(arquivo, sheet_name='contratos')
    cod_num_contrato = {}
    for _idx, row in df[['COD_CONTRATO', 'NUM_CONTRATO']].iterrows():
        cod_num_contrato.update({int(row['COD_CONTRATO']): row['NUM_CONTRATO']})
    return cod_num_contrato

In [ ]:
def listar_planilhas(extensao='xlsx'):
    search = "input/*.{}".format(extensao)
    files = glob.glob(search)
    files.sort()
    return files

#### Ajustando valores float em tabelas com duas casas decimais

In [ ]:
pd.options.display.float_format = "{:.2f}".format

In [ ]:
CONTRATO = obter_codigo_numero_contrato()

In [ ]:
OS = obter_codigo_nome_os()

In [ ]:
PREFEITO = {
    2001: 'Cesar Maia',
    2002: 'Cesar Maia',
    2003: 'Cesar Maia',
    2004: 'Cesar Maia',
    2005: 'Cesar Maia',
    2006: 'Cesar Maia',
    2007: 'Cesar Maia',
    2008: 'Cesar Maia',
    2009: 'Eduardo Paes',
    2010: 'Eduardo Paes',
    2011: 'Eduardo Paes',
    2012: 'Eduardo Paes',
    2013: 'Eduardo Paes',
    2014: 'Eduardo Paes',
    2015: 'Eduardo Paes',
    2016: 'Eduardo Paes',
    2017: 'Marcelo Crivella',
    2018: 'Marcelo Crivella',
    2019: 'Marcelo Crivella',
    2020: 'Marcelo Crivella',
    2021: 'A definir'
}

#### Criação de Dataframes

In [ ]:
dados_prefeito = []
for ano, nome in PREFEITO.items():
    dados_prefeito.append([ano, nome])
df_prefeito = pd.DataFrame(columns=['ano', 'prefeito'], data=dados_prefeito)

In [ ]:
dff = pd.DataFrame()
for arquivo in listar_arquivos():
    os_id, contrato_id = obter_os_contrato(arquivo)
    dados = ler_json(arquivo)
    df = pd_json.json_normalize(dados)
    df['os_id'] = os_id
    df['os_nome'] = OS[os_id]
    df['contrato_id'] = contrato_id
    df['contrato_numero'] = CONTRATO[contrato_id]
    df = pd.merge(left=df, right=df_prefeito, how='left', left_on='ref_ano', right_on='ano')
    dff = dff.append(df)

In [ ]:
dff.head(5)

#### Gastos anuais declarados

In [ ]:
agregacoes = {
    'valor': 'sum',
    'os_id': pd.Series.nunique
}
df_gastos_anuais = dff.groupby('ref_ano').agg(agregacoes)

In [ ]:
df_gastos_anuais.to_excel('output/graficos/gastos-anuais-raw.xlsx')

#### Gastos anuais declarados por OS

In [ ]:
agregacoes = {
    'valor': 'sum'
}
df_gastos_anuais_por_os = dff.groupby(['ref_ano', 'os_nome']).agg(agregacoes)

In [ ]:
df_gastos_anuais_por_os.to_excel('output/graficos/gastos-anuais-por-os-raw.xlsx')

#### Gastos anuais por Prefeito

In [ ]:
agregacoes = {
    'valor': 'sum'
}
df_gastos_anuais_por_prefeito = dff.groupby(['ref_ano', 'prefeito']).agg(agregacoes)

In [ ]:
df_gastos_anuais_por_prefeito.to_excel('output/graficos/gastos-anuais-por-prefeito-raw.xlsx')

#### Gastos Segmentados por tipo de despesas

In [ ]:
agregacoes = {
    'valor': 'sum'
}
df_gastos_os_por_despesa = dff[dff['nivel'] == '1'].groupby(['os_nome', 'despesa']).agg(agregacoes)

In [ ]:
df_gastos_os_por_despesa.to_excel('output/graficos/gastos-declarados-os-por-despesas-raw.xlsx')

#### Quantidade de contratos por OS

In [ ]:
agregacoes = {
    'contrato_id': pd.Series.nunique,
    'valor': 'sum'
}
df_qtd_contrato_por_os = dff.groupby(['ref_ano', 'os_nome']).agg(agregacoes)

In [ ]:
df_qtd_contrato_por_os.to_excel('output/graficos/quantidade-contratos-os-por-ano-raw.xlsx')

In [ ]:
df_qtd_contrato_por_os

In [ ]:
# dff[(dff['os_id'] == 258) & (dff['valor'] > 0)]